<a href="https://colab.research.google.com/github/creativesar/Guess-the-number-Computer-/blob/main/Guess_the_number_by_computer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import time
import os

def clear_screen():
    os.system('cls' if os.name == 'nt' else 'clear')

def display_header():
    header = """
    ╔═══════════════════════════════════════╗
    ║         NUMBER MASTERMIND GAME        ║
    ╚═══════════════════════════════════════╝
    """
    print(header)

def get_difficulty():
    display_header()
    print("Choose your difficulty level:")
    print("1. Easy   (3-digit number, 12 attempts)")
    print("2. Medium (4-digit number, 10 attempts)")
    print("3. Hard   (5-digit number, 8 attempts)")

    while True:
        try:
            choice = int(input("Enter your choice (1-3): "))
            if choice == 1:
                return 3, 12  # 3 digits, 12 attempts
            elif choice == 2:
                return 4, 10  # 4 digits, 10 attempts
            elif choice == 3:
                return 5, 8   # 5 digits, 8 attempts
            else:
                print("Please enter a number between 1 and 3.")
        except ValueError:
            print("Please enter a valid number.")

def generate_secret_code(length):
    # Generate a random code with the specified number of digits
    return [random.randint(0, 9) for _ in range(length)]

def get_hint(secret_code):
    # Provide a random hint about the secret code
    hint_type = random.randint(1, 4)

    if hint_type == 1:
        # Sum of digits
        return f"The sum of all digits is {sum(secret_code)}."
    elif hint_type == 2:
        # Reveal a random position
        pos = random.randint(0, len(secret_code)-1)
        return f"The digit at position {pos+1} is {secret_code[pos]}."
    elif hint_type == 3:
        # Largest digit
        return f"The largest digit in the code is {max(secret_code)}."
    else:
        # Smallest digit
        return f"The smallest digit in the code is {min(secret_code)}."

def evaluate_guess(guess, secret_code):
    # Count exact matches (correct digit in correct position)
    exact_matches = sum(1 for i in range(len(secret_code)) if guess[i] == secret_code[i])

    # Count digit matches (correct digit but wrong position)
    guess_counts = [0] * 10
    secret_counts = [0] * 10

    for i in range(len(secret_code)):
        if guess[i] != secret_code[i]:  # Don't count exact matches twice
            guess_counts[guess[i]] += 1
            secret_counts[secret_code[i]] += 1

    digit_matches = sum(min(guess_counts[i], secret_counts[i]) for i in range(10))

    return exact_matches, digit_matches

def guess_the_number():
    high_score = 0
    games_played = 0

    while True:
        clear_screen()
        digits, max_attempts = get_difficulty()
        secret_code = generate_secret_code(digits)
        attempts = 0
        hints_used = 0

        clear_screen()
        display_header()
        print(f"I've generated a {digits}-digit code. Try to guess it!")
        print(f"You have {max_attempts} attempts.")
        print(f"Current high score: {high_score}")
        print("\n🔍 MASTERMIND RULES:")
        print("- After each guess, you'll get feedback:")
        print("  🎯 = correct digit in correct position")
        print("  ⭕ = correct digit in wrong position")

        while attempts < max_attempts:
            try:
                print("\nOptions:")
                print("- Enter a number to make a guess")
                print("- Type 'hint' for a clue (costs an attempt)")
                print("- Type 'show' to reveal the answer (ends the game)")

                user_input = input(f"\nAttempt {attempts+1}/{max_attempts}: ").lower()

                if user_input == 'hint':
                    attempts += 1
                    hints_used += 1
                    print(f"💡 HINT: {get_hint(secret_code)}")
                    print(f"That cost you an attempt. {max_attempts - attempts} attempts remaining.")
                    continue

                elif user_input == 'show':
                    print(f"The secret code was: {''.join(map(str, secret_code))}")
                    print("Game over! Better luck next time.")
                    break

                # Validate guess
                if not user_input.isdigit() or len(user_input) != digits:
                    print(f"Please enter a {digits}-digit number.")
                    continue

                guess = [int(digit) for digit in user_input]
                attempts += 1

                # Dramatic pause for effect
                print("Checking your guess", end="")
                for _ in range(3):
                    time.sleep(0.3)
                    print(".", end="", flush=True)
                print()

                # Evaluate the guess
                exact, partial = evaluate_guess(guess, secret_code)

                # Display feedback
                feedback = "🎯" * exact + "⭕" * partial
                print(f"Result: {feedback}")

                if exact == digits:
                    print("\n🎉 CONGRATULATIONS! 🎉")
                    print(f"You cracked the code in {attempts} attempts!")

                    # Calculate score - more points for harder difficulty and fewer attempts
                    score = (digits * 100) - (attempts * 10) - (hints_used * 20)
                    print(f"You earned {score} points!")

                    if score > high_score:
                        print(f"NEW HIGH SCORE! Previous: {high_score}, New: {score}")
                        high_score = score

                    break
                else:
                    print(f"You have {exact} correct digits in the right position.")
                    print(f"You have {partial} correct digits in the wrong position.")

                # Inform player of remaining attempts
                remaining = max_attempts - attempts
                if remaining > 0:
                    print(f"You have {remaining} attempts left.")

                    # Give encouragement when running low on attempts
                    if remaining <= 2:
                        print("You're running out of attempts! Focus!")
                        # Offer a free hint when down to last attempt
                        if remaining == 1 and hints_used == 0:
                            print("Here's a free hint to help you:")
                            print(f"💡 HINT: {get_hint(secret_code)}")

            except ValueError:
                print("Please enter a valid number or command.")

        if attempts >= max_attempts and exact != digits:
            print(f"\n😢 GAME OVER! The code was {''.join(map(str, secret_code))}.")

        games_played += 1

        # Ask if player wants to play again
        play_again = input("\nDo you want to play again? (yes/no): ").lower()
        if play_again not in ["yes", "y"]:
            print("\nThanks for playing! Final stats:")
            print(f"Games played: {games_played}")
            print(f"High score: {high_score}")
            print("\nGoodbye! 👋")
            break

if __name__ == "__main__":
    guess_the_number()


    ╔═══════════════════════════════════════╗
    ║         NUMBER MASTERMIND GAME        ║
    ╚═══════════════════════════════════════╝
    
Choose your difficulty level:
1. Easy   (3-digit number, 12 attempts)
2. Medium (4-digit number, 10 attempts)
3. Hard   (5-digit number, 8 attempts)
